In [1]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.parquet


In [6]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [11]:
df = con.execute("SELECT * FROM read_parquet('/content/drive/MyDrive/review_dataset/w_review_train.parquet')").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [12]:
df.shape

(40000, 2)

In [13]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [14]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [15]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [16]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [18]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [19]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.05 s, sys: 55.6 ms, total: 2.1 s
Wall time: 2.3 s


In [20]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [26]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_parquet('/content/drive/MyDrive/review_dataset/w_review_train.parquet') WHERE {where_clause}").fetchdf()

CPU times: user 1.88 s, sys: 48.1 ms, total: 1.93 s
Wall time: 1.94 s


In [ ]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [27]:
coffee_reviews_df.shape

(4981, 2)

In [28]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [29]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [30]:
!ollama pull gemma3:270m

In [31]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [32]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ



In [33]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T16:10:34.688776867Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [34]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [35]:
llm = OllamaLLM(model="gemma3:270m")

In [36]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊



In [37]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [38]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [39]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [40]:
chain = prompt | llm

In [41]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [42]:
response

'```python\ndef get_sum(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  total = 0\n  for number in numbers:\n    total += number\n  return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = get_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def get_sum(numbers):`**:  กำหนดชื่อฟังก์ชัน `get_sum` ซึ่งจะถูกเรียกใช้เมื่อมีค่า `numbers` เป็นค่าเริ่มต้น\n2.  **`total = 0`**:  ตั้งค่าค่า `total` เป็น 0 เพื่อเก็บผลรวม\n3.  **`for number in numbers:`**:  ใช้ `for` 루ก `numbers`  เพื่อวนลูปผ่านแต่ละตัวเลขใน `numbers`\n4.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (แทนด้วย `number` ในตัวถัดไป)\n5.  **`return total`**:  คืนค่า `total`  เพื่อเก็บผลรวมทั้งหมด\n6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวแปร `numbers`  ที่มีค่า 1, 2, 3, 4, 5, 6, 7, 8, 9, 10\n7.  **`result = get_sum(numbers)`**:  เรียกใช้ฟังก์ชัน `get_sum`  เพื่อหาผลรวมของ `numbers`\n8.  **`print(result)`**:  แสดงผลลัพธ์ของฟังก์ชัน `get_sum`  เป

In [43]:
from IPython.display import display, Markdown

In [44]:
display(Markdown(response))

```python
def get_sum(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  total = 0
  for number in numbers:
    total += number
  return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = get_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`def get_sum(numbers):`**:  กำหนดชื่อฟังก์ชัน `get_sum` ซึ่งจะถูกเรียกใช้เมื่อมีค่า `numbers` เป็นค่าเริ่มต้น
2.  **`total = 0`**:  ตั้งค่าค่า `total` เป็น 0 เพื่อเก็บผลรวม
3.  **`for number in numbers:`**:  ใช้ `for` 루ก `numbers`  เพื่อวนลูปผ่านแต่ละตัวเลขใน `numbers`
4.  **`total += number`**:  เพิ่มค่า `number` เข้าไปใน `total`  (แทนด้วย `number` ในตัวถัดไป)
5.  **`return total`**:  คืนค่า `total`  เพื่อเก็บผลรวมทั้งหมด
6.  **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  สร้างตัวแปร `numbers`  ที่มีค่า 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
7.  **`result = get_sum(numbers)`**:  เรียกใช้ฟังก์ชัน `get_sum`  เพื่อหาผลรวมของ `numbers`
8.  **`print(result)`**:  แสดงผลลัพธ์ของฟังก์ชัน `get_sum`  เป็นตัวเลข

**การใช้งานโค้ด:**

1.  **ตัวอย่างการใช้งาน:**  เราสร้างตัวแปร `numbers` ที่มีค่า 1, 2, 3, 4, 5, 6, 7, 8, 9, 10  และเรียกใช้ฟังก์ชัน `get_sum`  เพื่อหาผลรวมของตัวเลขเหล่านี้
2.  **การแสดงผล:**  เราจะแสดงผลลัพธ์ของฟังก์ชัน `get_sum`  เป็นตัวเลข  `1, 2, 3, 4, 5, 6, 7, 8, 9, 10`  ซึ่งแสดงว่ามี 10 ตัวเลขที่รวมกันได้


In [45]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [46]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [47]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [48]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [49]:
parser_chain = StrOutputParser()

In [50]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [51]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [52]:
result

'ผลลัทธ์เป็นเครื่องมือสำคัญในการจัดการและควบคุมการดำเนินงานขององค์กรหรือธุรกิจ โดยมีวัตถุประสงค์หลักคือการสร้างสภาพแวดล้อมที่ให้การดำเนินงานที่ราบรื่นและมีประสิทธิภาพ โดยการนำเอาหลักการและแนวคิดจากผลลัทธ์มาปรับใช้กับสถานการณ์และเป้าหมายขององค์กร\n'

In [53]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [54]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [55]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [56]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [57]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [58]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [59]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [60]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [61]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 45.3 ms, sys: 4.95 ms, total: 50.2 ms
Wall time: 4.79 s


In [62]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ทอดมัน", "หัวปลีก", "อร่อย", "เบิ้ล", "ขนมไทย", "ขนมหวาน", "ฟรี", "เล็กน้อย"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "จัดไป",
    "รอ",
    "โต๊ะ",
    "ริมกระจก",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ฉีกน้ำตาล",
    "คน",
    "น้ำส้มกะเค้ก",
    "แจ่ม",
    "เมพพพ",
    "ซากอารยธรรมลาเต้",
    "เมพพพพ",
    "รสชาติ",
    "กลิ่น",
    "มวยรุ่นเดียวกะ Starbuck"
  ]
}
```



In [63]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [64]:
!ollama list

NAME                                  ID              SIZE      MODIFIED      
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    2 seconds ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    2 minutes ago    


In [65]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [66]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [67]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 118 ms, sys: 9.17 ms, total: 128 ms
Wall time: 16.9 s


In [68]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงประสบการณ์การทานอาหารที่ร้านอาหารแห่งหนึ่ง โดยมีประเด็นหลักๆ ดังนี้:

*   ร้านใหญ่มาก
*   ห้องน้ำอยู่ก่อนถึงห้องอาหาร
*   อาหารบางอย่างราคาแพงเกินไป (เช่น น้ำผึ้ง, พะแนงห่อไข่)
*   อาหารบางอย่างอร่อย (เช่น ทอดมันหัวปลี)
*   มีของหวานฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

หัวข้อหลักที่กล่าวถึงในรีวิวนี้ ได้แก่:

*   ขนาดร้าน
*   ราคาอาหาร
*   รสชาติอาหาร (ทั้งอร่อยและไม่น่าประทับใจ)
*   ของหวาน

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำสำคัญ 3 คำ ได้แก่:

1.  ราคา (เนื่องจากมีการกล่าวถึงราคาอาหารที่สูงเกินไป)
2.  รสชาติ (มีการกล่าวถึงทั้งอาหารที่อร่อยและไม่น่าประทับใจ)
3.  ขนาด (ร้านใหญ่)

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "รสชาติ", "ขนาด"]
}
```

หวังว่าคำตอบนี้จะเป็นประ

In [69]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [70]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [71]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [72]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 107 ms, sys: 10.4 ms, total: 117 ms
Wall time: 10.7 s


In [73]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาด']

In [74]:
results[1]['text']['keywords']

['วาวี', 'ลาเต้', 'บรรยากาศ']

In [75]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [76]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 161 ms, sys: 11.4 ms, total: 172 ms
Wall time: 15 s


In [77]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [78]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [79]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [80]:
inputs = [{"review": r} for r in sample_reviews]

In [81]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 21 ms, sys: 592 µs, total: 21.6 ms
Wall time: 2.42 s


In [82]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [83]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[0:200].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]

BATCH_SIZE = 8

results_100 = []

In [ ]:
from tqdm import tqdm

for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords:  32%|███▏      | 8/25 [01:22<02:54, 10.27s/it]

In [ ]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part1.csv", index=False, encoding='utf-8-sig')

In [ ]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[200:400].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:23<00:00, 10.53s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part2.csv", index=False, encoding='utf-8-sig')

In [ ]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[400:600].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:16<00:00, 10.27s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part3.csv", index=False, encoding='utf-8-sig')

In [ ]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[600:800].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:19<00:00, 10.38s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part4.csv", index=False, encoding='utf-8-sig')

In [ ]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[800:1000].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:09<00:00,  9.96s/it]


In [ ]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part5.csv", index=False, encoding='utf-8-sig')

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 25/25 [04:13<00:00, 10.15s/it]


In [ ]:
for result in results_100[0:10]:
    print(result['text']['keywords'])

ช็อกโกแลต, เค้ก, ขนมหวาน
สนามบิน, ร้านอาหาร, บรรยากาศ
กาแฟ, เค้ก, บรรยากาศ
อาหารอิตาเลียน, บรรยากาศ, พิซซ่า
บรรยากาศ, ฟิวชั่น, ราคา
บรรยากาศ, ส้มตำ, รสชาติ
กาแฟ, เอ็กซ์เพรสโซ่, กลิ่น
วาฟเฟิล, กาแฟ, ไซรัป
กาแฟ, โปรโมชั่น, คนเยอะ
อาหารฝรั่ง, ชีสทอด, ฮันนี่โทส


In [ ]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [ ]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [ ]:
context_extract = context_prompt | llm3 | context_parser

In [ ]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [ ]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [ ]:
results_100[0]['text']['keywords'].split(',')

['ช็อกโกแลต', ' เค้ก', ' ขนมหวาน']

In [ ]:
context_inputs = []

for review_data in tqdm(results_100, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 400/400 [00:00<00:00, 149516.23it/s]


In [ ]:
len(context_inputs)

1200

In [ ]:
context_results = []
for i in tqdm(range(len(results_100)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_100[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 400/400 [29:58<00:00,  4.50s/it]


In [ ]:
results_100[:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา']}]

In [ ]:
sentiment_inputs = []

for review_data in tqdm(results_100, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 400/400 [00:00<00:00, 289912.15it/s]


In [ ]:
len(sentiment_inputs)

1200

In [ ]:
sentiment_inputs[:6]

[{'message': 'ช็อกโกแลต'},
 {'message': 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ'},
 {'message': 'ขนมอร่อย'},
 {'message': 'สนามบิน'},
 {'message': 'ร้านอาหารเล็กๆ'},
 {'message': 'บรรยากาศดีเหมือนอยู่ในท้องนา'}]

In [ ]:
for i in tqdm(range(len(results_100)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_100[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 400/400 [19:19<00:00,  2.90s/it]


In [ ]:
results_100[0:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย'],
  'sentiment': ['neutral', 'neutral', 'positive']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา'],
  'sentiment': ['neutral', 'neutral', 'positive']}]

In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_100
])

In [ ]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')